# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [ ]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [ ]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

In [ ]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [ ]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [ ]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [ ]:
np.linalg.matrix_rank(a1)

In [ ]:
l, u = diy_lu(a1)

print(l, u)

### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [ ]:
def minor_check(b):
    n = b.shape[0]
    for i in range(n, 1, -1):
        det = np.linalg.det(b)
        if det ==  0:
            k = False
            break
        else : 
            k = True
        b = b[:(i-1),:(i-1)]
    return k
l = minor_check(a), minor_check(a1)
print(l)


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [22]:
def diy_lu_mod(a):
    N = a.shape[0]

    u = a.copy()
    L = np.eye(N)
    det = np.linalg.det(a)
    if det == 0:
        return False
    else:
        for j in range(N-1):
            dj = np.linalg.det(u[:j,:j])
            while dj == 0:
                i = j+1
                while i < N-1:
                    u[:,[i,i+1]] = u[:,[i+1,i]]
                    print(u[:,[i,i+1]])
                    i +=1
                dj = np.linalg.det(u[:j,:j])
                print(dj)
            lam = np.eye(N)
            gamma = u[j+1:, j] / u[j, j]
            lam[j+1:, j] = -gamma
            u = lam @ u

            lam[j+1:, j] = gamma
            L = L @ lam
    return L, u


L, u = diy_lu_mod(a)
print(L, "\n")
print(u, "\n")

L, u = diy_lu_mod(a1)
print(L, "\n")
print(u, "\n")



[  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
0.0
[[ 3.     3.   ]
 [-2.118 -1.929]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
[[ 3.     3.   ]
 [-2.25  -1.929]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
0.0
[[ 3.     3.   ]
 [-2.25  -2.118]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
[[ 3.     3.   ]
 [-1.929 -2.118]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
0.0
[[ 3.     3.   ]
 [-1.929 -2.25 ]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
[[ 3.     3.   ]
 [-2.118 -2.25 ]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
0.0
[[ 3.     3.   ]
 [-2.118 -1.929]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
[[ 3.     3.   ]
 [-2.25  -1.929]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
0.0
[[ 3.     3.   ]
 [-2.25  -2.118]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]
 [  -inf   -inf]]
[[